# Module 2: Document Processing & Chunking
  
**Level:** Beginner to Intermediate  
**Prerequisites:** Module 1 completed

---

## Learning Objectives

By the end of this module, you will be able to:

- Explain why chunking is necessary in RAG systems
- Implement basic chunking strategies from scratch
- Choose appropriate chunk sizes for different use cases
- Understand the trade-offs between different chunking approaches
- Preserve and use document metadata effectively
- Handle different document formats (text, PDF)

---

# 1. Why Chunking Matters

## 1.1 The Problem with Whole Documents

Imagine you have a 50-page legal contract and someone asks: *"What is the termination notice period?"*

**Problems with using the whole document:**

1. **Too long for context windows**
   - Most LLMs have limited context (4K-128K tokens)
   - A 50-page document might be 20,000+ tokens
   - Can't fit multiple documents in one prompt

2. **Poor retrieval precision**
   - Embedding a whole document loses specificity
   - Can't pinpoint exact relevant section
   - Answer might be buried in irrelevant content

3. **Inefficient and costly**
   - Processing entire documents is slow
   - Expensive in terms of tokens/API costs
   - Wastes context window space

## 1.2 How Chunking Solves This

**Chunking = Breaking documents into smaller, meaningful pieces**

**Benefits:**
- Each chunk fits in context window
- More precise retrieval (find exact relevant section)
- Better embeddings (more specific semantic meaning)
- Faster and cheaper processing

**Example:**
```
50-page contract
      ↓
Split into 100 chunks (500 words each)
      ↓
Embed each chunk separately
      ↓
Retrieve only the 3 most relevant chunks
```

---

# 2. Understanding Chunk Size

The most important decision in chunking: **How big should chunks be?**

## 2.1 The Chunk Size Trade-off

### Small Chunks (100-300 tokens)

**Pros:**
- Very precise retrieval
- Specific answers to specific questions
- Less noise in retrieved context

**Cons:**
- May lose important surrounding context
- Might split related information
- Need to retrieve more chunks to get full picture

**Best for:** FAQ systems, specific fact lookup

---

### Medium Chunks (300-600 tokens)

**Pros:**
- Good balance of precision and context
- Usually keeps related information together
- Most versatile option

**Cons:**
- Middle ground = not optimized for any specific case

**Best for:** Most general RAG use cases (recommended starting point)

---

### Large Chunks (600-1000+ tokens)

**Pros:**
- Preserves more context
- Better for complex, interconnected information
- Fewer chunks to manage

**Cons:**
- Less precise retrieval
- More irrelevant information included
- May exceed context limits with multiple chunks

**Best for:** research papers, narrative documents

## 2.2 Rule of Thumb

**Start with 400-500 tokens (roughly 300-400 words)**

This is a good default for most use cases. You can adjust based on:
- Document type (technical docs → smaller, narratives → larger)
- Query types (specific facts → smaller, summaries → larger)
- Testing and evaluation results

**Remember:** There's no perfect size - it depends on your use case!

---

# 3. Chunking Strategies

Let's build chunking strategies from scratch to understand how they work.

## Setup: Sample Document

First, let's create a sample document to work with.

In [ ]:
# Sample document about Python programming
sample_document = """
Python is a high-level, interpreted programming language known for its simplicity and readability. 
It was created by Guido van Rossum and first released in 1991. Python supports multiple programming 
paradigms including procedural, object-oriented, and functional programming.

One of Python's key strengths is its extensive standard library, which provides tools for many common 
programming tasks. The language emphasizes code readability with its use of significant indentation. 
Python's syntax allows programmers to express concepts in fewer lines of code compared to languages 
like C++ or Java.

Python is widely used in web development, data science, artificial intelligence, scientific computing, 
and automation. Popular frameworks include Django and Flask for web development, NumPy and Pandas for 
data analysis, and TensorFlow and PyTorch for machine learning.

The Python Package Index (PyPI) hosts hundreds of thousands of third-party packages that extend Python's 
capabilities. Installation is simple using pip, Python's package installer. The active community contributes 
to a rich ecosystem of libraries and frameworks.

Python continues to be one of the most popular programming languages worldwide. Its beginner-friendly nature 
makes it ideal for education, while its powerful features support professional software development and research.
"""

print(f"Document length: {len(sample_document)} characters")
print(f"Document length: {len(sample_document.split())} words")
print(f"\nFirst 200 characters:\n{sample_document[:200]}...")

## 3.1 Fixed-Size Chunking (Character-Based)

**Simplest approach:** Split text every N characters.

**How it works:**
1. Set a chunk size (e.g., 200 characters)
2. Split the text at every 200 characters
3. Optionally add overlap between chunks

In [ ]:
def chunk_by_characters(text, chunk_size=200, overlap=50):
    """
    Split text into chunks of specified character length.
    
    Args:
        text: The text to chunk
        chunk_size: Number of characters per chunk
        overlap: Number of characters to overlap between chunks
    
    Returns:
        List of text chunks
    """
    chunks = []
    start = 0
    
    while start < len(text):
        # Get chunk from start to start + chunk_size
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        
        # Move start position (with overlap)
        start += chunk_size - overlap
    
    return chunks

# Test it
chunks = chunk_by_characters(sample_document, chunk_size=200, overlap=50)

print(f"Number of chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks[:3], 1):  # Show first 3 chunks
    print(f"Chunk {i} ({len(chunk)} chars):")
    print(chunk)
    print("-" * 80)

### 💭 Observations

**Notice the problems with character-based chunking:**
- Chunks split in the middle of words
- Chunks split in the middle of sentences
- No respect for natural boundaries

**This is why we need smarter chunking strategies!**

## 3.2 Fixed-Size Chunking (Word-Based)

**Better approach:** Split by words instead of characters.

**How it works:**
1. Split text into words
2. Group words into chunks of N words
3. Join words back into text

In [ ]:
def chunk_by_words(text, chunk_size=50, overlap=10):
    """
    Split text into chunks of specified word count.
    
    Args:
        text: The text to chunk
        chunk_size: Number of words per chunk
        overlap: Number of words to overlap between chunks
    
    Returns:
        List of text chunks
    """
    # Split text into words
    words = text.split()
    chunks = []
    start = 0
    
    while start < len(words):
        # Get chunk of words
        end = start + chunk_size
        chunk_words = words[start:end]
        
        # Join words back into text
        chunk = ' '.join(chunk_words)
        chunks.append(chunk)
        
        # Move start position (with overlap)
        start += chunk_size - overlap
    
    return chunks

# Test it
chunks = chunk_by_words(sample_document, chunk_size=50, overlap=10)

print(f"Number of chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks[:3], 1):  # Show first 3 chunks
    print(f"Chunk {i} ({len(chunk.split())} words):")
    print(chunk)
    print("-" * 80)

### 💭 Better, but still not perfect!

**Improvements:**
- No more mid-word splits ✅
- Easier to control chunk size ✅

**Remaining issues:**
- Still splits in the middle of sentences ❌
- Doesn't respect paragraph boundaries ❌

**Let's fix this with sentence-based chunking!**

## 3.3 Sentence-Based Chunking

**Better approach:** Split by sentences, then group into chunks.

**How it works:**
1. Split text into sentences
2. Group sentences until reaching target chunk size
3. Preserves sentence boundaries

In [ ]:
def chunk_by_sentences(text, max_chunk_size=500):
    """
    Split text into chunks by sentences, keeping sentences intact.
    
    Args:
        text: The text to chunk
        max_chunk_size: Maximum characters per chunk
    
    Returns:
        List of text chunks
    """
    # Simple sentence splitting (split on . ! ?)
    import re
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Check if adding this sentence would exceed max size
        if len(current_chunk) + len(sentence) > max_chunk_size and current_chunk:
            # Save current chunk and start new one
            chunks.append(current_chunk.strip())
            current_chunk = sentence
        else:
            # Add sentence to current chunk
            current_chunk += " " + sentence if current_chunk else sentence
    
    # Don't forget the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# Test it
chunks = chunk_by_sentences(sample_document, max_chunk_size=400)

print(f"Number of chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i} ({len(chunk)} chars):")
    print(chunk)
    print("-" * 80)

### ✅ Much better!

**Advantages:**
- Complete sentences ✅
- More natural reading ✅
- Better semantic coherence ✅

**This is the recommended approach for most use cases!**

## 3.4 Paragraph-Based Chunking

**For structured documents:** Use natural paragraph breaks.

**How it works:**
1. Split by paragraph (double newlines)
2. Each paragraph becomes a chunk (or combine small ones)

In [ ]:
def chunk_by_paragraphs(text, min_chunk_size=100):
    """
    Split text by paragraphs (double newlines).
    
    Args:
        text: The text to chunk
        min_chunk_size: Minimum characters per chunk (combine small paragraphs)
    
    Returns:
        List of text chunks
    """
    # Split by double newlines (paragraph separator)
    paragraphs = text.split('\n\n')
    
    chunks = []
    current_chunk = ""
    
    for para in paragraphs:
        para = para.strip()
        if not para:
            continue
            
        # If paragraph is too small, combine with next
        if len(para) < min_chunk_size:
            current_chunk += "\n\n" + para if current_chunk else para
        else:
            # Save previous chunk if exists
            if current_chunk:
                chunks.append(current_chunk.strip())
            # Start new chunk with this paragraph
            current_chunk = para
    
    # Don't forget the last chunk
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks

# Test it
chunks = chunk_by_paragraphs(sample_document, min_chunk_size=100)

print(f"Number of chunks: {len(chunks)}\n")
for i, chunk in enumerate(chunks, 1):
    print(f"Chunk {i} ({len(chunk)} chars):")
    print(chunk)
    print("-" * 80)

### When to use paragraph chunking:

**Good for:**
- Well-structured documents (articles, reports, documentation)
- Documents where paragraphs are coherent units
- Markdown or formatted text

**Not good for:**
- Unstructured text (chat logs, transcripts)
- Very long paragraphs
- Documents without clear paragraph breaks

---

# 4. Chunk Overlap: Why It Matters

## 4.1 The Problem with No Overlap

**Consider this example:**

```
Chunk 1: "...the contract term is 12 months."
Chunk 2: "The renewal process requires 30 days notice..."
```

**Query:** "What is the contract term and renewal notice period?"

**Problem:** The answer is split across two chunks with no connection!

## 4.2 The Solution: Overlap

**With overlap:**
```
Chunk 1: "...the contract term is 12 months. The renewal process..."
Chunk 2: "...12 months. The renewal process requires 30 days notice..."
```

Now Chunk 2 has context from Chunk 1!

## 4.3 How Much Overlap?

**Rule of thumb:** 10-20% of chunk size

- Chunk size 500 words → Overlap 50-100 words
- Chunk size 1000 tokens → Overlap 100-200 tokens

**Trade-off:**
- More overlap = Better context, but more storage and processing
- Less overlap = Less storage, but may lose connections

---

# 5. Adding Metadata to Chunks

## 5.1 Why Metadata Matters

**Metadata = Information about the chunk**

**Useful metadata:**
- Source document name
- Page number
- Chunk index
- Section/chapter title
- Creation date
- Author
- Document type

**Why it's important:**
- **Source attribution:** Tell users where information came from
- **Filtering:** Retrieve only from specific sources or dates
- **Navigation:** Link back to original document
- **Verification:** Users can verify information

## 5.2 Implementing Metadata

In [ ]:
def chunk_with_metadata(text, source_name, chunk_size=50, overlap=10):
    """
    Create chunks with metadata.
    
    Args:
        text: The text to chunk
        source_name: Name of the source document
        chunk_size: Number of words per chunk
        overlap: Number of words to overlap
    
    Returns:
        List of dictionaries with 'text' and 'metadata'
    """
    # Get basic chunks
    words = text.split()
    chunks = []
    start = 0
    chunk_index = 0
    
    while start < len(words):
        end = start + chunk_size
        chunk_words = words[start:end]
        chunk_text = ' '.join(chunk_words)
        
        # Create chunk with metadata
        chunk_with_meta = {
            'text': chunk_text,
            'metadata': {
                'source': source_name,
                'chunk_index': chunk_index,
                'chunk_size': len(chunk_words),
                'char_count': len(chunk_text),
                'start_word': start,
                'end_word': end
            }
        }
        
        chunks.append(chunk_with_meta)
        start += chunk_size - overlap
        chunk_index += 1
    
    return chunks

# Test it
chunks = chunk_with_metadata(
    sample_document, 
    source_name="python_intro.txt",
    chunk_size=50,
    overlap=10
)

print(f"Total chunks: {len(chunks)}\n")
print("First chunk with metadata:")
print("=" * 80)
print(f"Text: {chunks[1]['text'][:200]}...")
print(f"\nMetadata:")
for key, value in chunks[1]['metadata'].items():
    print(f"  {key}: {value}")

---

# 6. Practical Example: Loading and Chunking a Real Document

## 6.1 Loading Text Files

In [ ]:
def load_and_chunk_text_file(file_path, chunk_size=500, overlap=50):
    """
    Load a text file and chunk it.
    
    Args:
        file_path: Path to the text file
        chunk_size: Characters per chunk
        overlap: Character overlap between chunks
    
    Returns:
        List of chunks with metadata
    """
    import os
    
    # Read the file
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Get file metadata
    file_name = os.path.basename(file_path)
    file_size = os.path.getsize(file_path)
    
    # Chunk the text
    chunks = chunk_by_sentences(text, max_chunk_size=chunk_size)
    
    # Add metadata to each chunk
    chunks_with_metadata = []
    for i, chunk in enumerate(chunks):
        chunks_with_metadata.append({
            'text': chunk,
            'metadata': {
                'source': file_name,
                'file_path': file_path,
                'file_size': file_size,
                'chunk_index': i,
                'total_chunks': len(chunks)
            }
        })
    
    return chunks_with_metadata

# Example usage (create a sample file first)
sample_file_path = 'sample_document.txt'
with open(sample_file_path, 'w', encoding='utf-8') as f:
    f.write(sample_document)

# Load and chunk
chunks = load_and_chunk_text_file(sample_file_path, chunk_size=400)

print(f"Loaded and chunked: {chunks[0]['metadata']['source']}")
print(f"Total chunks: {len(chunks)}")
print(f"\nChunk 1:")
print(chunks[0]['text'])

In [ ]:
chunks[2]['metadata']

## 6.2 Loading PDFs (Using PyPDF2)

In [ ]:
# Install PyPDF2 if needed
!pip install -q PyPDF2

In [ ]:
def load_and_chunk_pdf(file_path, chunk_size=500):
    """
    Load a PDF file and chunk it.
    
    Args:
        file_path: Path to the PDF file
        chunk_size: Characters per chunk
    
    Returns:
        List of chunks with metadata (including page numbers)
    """
    import PyPDF2
    import os
    
    chunks_with_metadata = []
    file_name = os.path.basename(file_path)
    
    # Open PDF
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        num_pages = len(pdf_reader.pages)
        
        # Process each page
        for page_num in range(num_pages):
            # Extract text from page
            page = pdf_reader.pages[page_num]
            text = page.extract_text()
            
            # Chunk the page text
            page_chunks = chunk_by_sentences(text, max_chunk_size=chunk_size)
            
            # Add metadata to each chunk
            for chunk_idx, chunk in enumerate(page_chunks):
                chunks_with_metadata.append({
                    'text': chunk,
                    'metadata': {
                        'source': file_name,
                        'page': page_num + 1,  # 1-indexed
                        'total_pages': num_pages,
                        'chunk_on_page': chunk_idx,
                    }
                })
    
    return chunks_with_metadata

# Example (you would use this with a real PDF file)
print("PDF loading function ready!")
print("\nUsage:")
print("chunks = load_and_chunk_pdf('your_document.pdf', chunk_size=500)")

In [ ]:
# You can test the load_and_chunk function with the code below

# chunks = load_and_chunk_pdf("Retrieval-Augmented_Generation_RAG.pdf", chunk_size=500)

# print(f"Total chunks: {len(chunks)}\n")

# for i, c in enumerate(chunks[:5]):  # show first 5 chunks
#     print(f"CHUNK {i+1}")
#     print("TEXT:", c["text"][:200])  # print first 200 chars
#     print("META:", c["metadata"])
#     print("-"*40)


---

# 7. Choosing the Right Strategy

1. **Start with sentence-based chunking (400-500 chars)** - works for 80% of cases
2. **Use paragraph-based for well-structured documents**
3. **Increase chunk size** if answers need more context
4. **Decrease chunk size** if you need more precise retrieval
5. **Always use some overlap** (10-20%) to maintain context
6. **Test and iterate** - there's no perfect size for all cases

---

# 8. Summary

## Key Takeaways

1. **Chunking is essential** for RAG - whole documents don't work well

2. **Chunk size matters:**
   - Small chunks (100-300 tokens): Precise but may lose context
   - Medium chunks (300-600 tokens): Best starting point
   - Large chunks (600-1000 tokens): More context but less precise

3. **Chunking strategies:**
   - Character-based: Simple but breaks words/sentences
   - Word-based: Better, but still breaks sentences
   - Sentence-based: **Recommended for most cases**
   - Paragraph-based: Good for structured documents

4. **Overlap is important:**
   - Maintains context between chunks
   - 10-20% overlap is typical

5. **Metadata is crucial:**
   - Enables source attribution
   - Allows filtering and navigation
   - Builds trust and verifiability

6. **No perfect solution:**
   - Different documents need different strategies
   - Test and iterate to find what works
   - Start simple, then optimize

# 🎯 Practice Exercises

## Exercise 1: Chunking Different Document Types

### Task
Apply appropriate chunking strategies to different document types.

### Documents

You'll chunk 3 different types of documents:

**Document A: FAQ**
```
Q: What is the return policy?
A: Items can be returned within 30 days of purchase with original receipt.

Q: Do you offer international shipping?
A: Yes, we ship to over 50 countries worldwide. Shipping times vary by location.

Q: How do I track my order?
A: Use the tracking number sent to your email after shipment.
```

**Document B: Technical Documentation**
```
Installation Guide

Step 1: Download the installer from our website.
Extract the zip file to your desired location.

Step 2: Run setup.exe as administrator.
Follow the on-screen instructions.

Step 3: Configure your API key in the settings file.
The settings file is located at config/settings.json.
```

**Document C: Article**
```
The Future of Renewable Energy

Solar and wind power have seen tremendous growth in recent years. As technology improves
and costs decrease, renewable energy becomes increasingly competitive with fossil fuels.

Energy storage solutions are critical for renewable adoption. Battery technology advances
enable better grid management and reliability. This addresses the intermittent nature of
solar and wind power.

Policy support and public awareness continue to drive the transition. Many countries have
set ambitious renewable energy targets for the coming decades.
```

### Instructions

For each document:
1. Choose the most appropriate chunking strategy (character, word, sentence, paragraph, or custom)
2. Explain why you chose that strategy
3. Implement the chunking
4. Show the resulting chunks

### Template

```python
# Document A: FAQ
strategy_A = "?"  # Your choice
reason_A = "?"    # Why this strategy?
chunks_A = ?      # Your implementation

# Document B: Technical Documentation
strategy_B = "?"
reason_B = "?"
chunks_B = ?

# Document C: Article
strategy_C = "?"
reason_C = "?"
chunks_C = ?
```


## What's Next?

In **Module 3: Embeddings Deep Dive**, you'll learn:
- How embedding models work
- Generating embeddings for your chunks
- Choosing the right embedding model
- Understanding vector similarity

Get ready to turn your text chunks into vectors! 🎯